In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('barkbeetle_dataset.csv')

# reestablish timestamps
data['timestamp'] = pd.to_datetime(data['timestamp'])

# remove pre 2006 and post feb2020 
data = data[data['timestamp'].isin(pd.date_range(start='2006-01-01', end='2020-09-30', freq='M'))]

data = data[['fdist_id', 'year', 'forest_ownership' , 'timeframe', 'RRK', 'TM0', 'demolition_wood', 'infested_wood']]
data.to_excel('input.xlsx', index=False)

Den folgenden Codeblock ausführen und dann die Excel-Datei mit den bisherigen Beobachtungen als 'input.xlsx' sowie das Modell als 'final_model.pkl' zum Hochladen auswählen.

In [ ]:
# Hochladen von input.xlsx und final_model.pkl

from google.colab import files
data_to_load = files.upload()

In [5]:
import pandas as pd
import numpy as np
import pickle
from pandas.tseries.offsets import MonthEnd

# Lese Datei mit bisherigen Beobachtungen
data = pd.read_excel(
    'input.xlsx',
    names=[
        'fdist_id',
        'year',
        'forest_ownership',
        'timeframe',
        'RRK',
        'TM0',
        'demolition_wood',
        'infested_wood'
    ]
)

data.head()

,fdist_id,year,forest_ownership,timeframe,RRK,TM0,demolition_wood,infested_wood
0,2501,2007,SW,06 Juni,72.526601,18.961279,0.0,5.0
1,2501,2007,NSW,06 Juni,72.526601,18.961279,0.0,0.0
2,2501,2007,SW,08 August,53.017506,18.506060,0.0,12.0
3,2501,2007,NSW,08 August,53.017506,18.506060,0.0,0.0
4,2501,2007,SW,10 Oktober-Dezember,138.329965,4.503592,0.0,2.0


# Einstellungen

Die Vorhersageparameter im folgenden Codeblock können von Ihnen frei verändert werden. Diese haben Einfluss auf die Berechnung der Vorhersagen. Der restliche Code kann dann direkt ausgeführt werden ohne die Notwendigkeit weiterer Anpassungen (es sei denn es ändern sich Forstbezirke o.Ä.).

Die Vorhersagen werden für drei Wetterszenarien getroffen, welche über die drei Parameter definiert sind. Die Angabe kann entweder über eine vierstellige Jahreszahl erfolgen oder über die Angabe eines Quantils im Bereich 1-99. Wird eine Zahl gewählt, dann werden die klimatischen Parameter aus diesem jahr übernommen. Bei Angabe eines Quantils, wird das jeweilige Quantil aus der gesamten Historie für den jeweiligen Monat berechnet und übernommen. Ein höherer Wert bedeutet hierbei, dass das Klima trockener und wärmer wird (d.h. bei 75 wird das 75%-Quantil der Temperatur und das 25%-Quantil des Niederschlags übernommen). Das 50%-Quantil entspricht dem Median.

Für das Wurf- und Bruchholz existiert ein weiterer Paramter, der nach dem gleichen Prinzip arbeitet. Dieser eine Wert wird für alle Wetterszenarien genutzt (falls Sie diesen gerne im Wetterszenario verbaut hätten, kann ich das gerne anpassen) 

Der letzte Parameter gibt an, bis wann die Vorhersage stattfinden soll. Prinzipiell wird empfohlen, nicht mehr als ein Jahr in Vorhersagen zu treffen. Da das Modell immer wieder seinen. Da der Output der Verohersage eines Monats als Input für die nächste Vorhersage genommen wird

In [51]:
# Vorhersageparameter

# Wetterszenarien
# Szenario warm/trocken
s_wt = 2018
# Szenario durchschnittlich
s_du = 50
# Szenario kalt/feucht
s_kf = 2012

# Annahme Wurf-/Bruchholz
s_wbh = 40

# Ende Vorhersagezeitraum ('YYYY-MM')
pred_end = '2021-05'

# Überprüfung des Inputs

In [68]:
# Überprüfe Integrität der Daten

# zähle Anzahl Warnungen
warn_count = 0

################################################################################
# Keine leeren Einträge

n_nan = data.isna().sum().sum()

if n_nan == 0:
    # Keine leeren Einträge in Beobachtungen
    pass
else:
    print(
        f'Warnung: {n_nan} leere Einträge in bisherigen Beobachtungen.\n'
        f'Dies beeinträchtigt unter Umständen die Qualität der Vorhersagen.\n'
    )
    warn_count += 1
    
################################################################################
# Keine Duplikate?
dup_rows = data.duplicated(
    ['fdist_id', 'forest_ownership', 'year', 'timeframe']
)

if dup_rows.any()==False:
    # Keine doppelten Einträge im Datansatz
    pass
else:    
    print(
        f'Warnung: Mehrere Einträge bestimmter Beobachtungen gefunden.\n'
        f'Folgende Zeilen werden gelöscht: {data[dup_rows]}.\n'
    )
    
    warn_count += 1
    
    data.drop_duplicates(
        ['fdist_id', 'forest_ownership', 'year', 'timeframe'], 
        inplace=True
    )

################################################################################
# Überprüfe, inwiefern aktuelle Werte für alle Bezirke vorliegen
# aktuellen Zeitraum bestimmen
max_year = data['year'].max()
max_timeframe = data[data['year'] == max_year]['timeframe'].max()

newest_data = data.loc[
    (data['year'] == max_year) & (data['timeframe'] == max_timeframe)
]

if newest_data.shape[0] == 106:
    # Aktuelle Werte für alle Forstbezirke gefunden
    newest_ids = newest_data['fdist_id'].unique()
else:
    # Vorhandene IDs
    newest_ids = newest_data['fdist_id'].unique()
    
    # Vorhandene IDs mit nur einer Eigentumsgruppe
    mo_ids = [
        ID for ID in newest_ids 
        if newest_data[newest_data['fdist_id']==ID].shape[0] == 1
    ]
    
    # Ausgabe
    print(
        f'Warnung: Aktuelle Werte für {len(newest_ids)}' 
        f'Forstbezirke gefunden.\n'
        f'In {len(mo_ids)} dieser Bezirke fehlt eine Eigentumsgruppe.\n'
        f'Nur vollständige Bezirke werden nachfolgend einbezogen.\n'
    )
    
    warn_count += 1
    
    # nur mit vollständigen Bezirken weiterrechnen
    newest_ids = [x for x in newest_ids if x not in mo_ids]


################################################################################    
# Überprüfe, ob für alle Bezirke die 7 letzten Beobachtungen existieren  

for ID in newest_ids:
    for fo in ['NSW', 'SW']:

        sub = data[
            (data['fdist_id'] == ID) & (data['forest_ownership'] == fo)
        ].copy()
        
        sub['ts'] = sub['year'].astype(str) + sub['timeframe'].map(
            lambda x: '-' + x.split(' ')[0])
        sub['ts'] = pd.to_datetime(sub['ts']) + MonthEnd()
        
        max_ts = str(max_year) + '-' + max_timeframe.split(' ')[0]
        max_ts = pd.to_datetime(max_ts) + MonthEnd()
        cur_ts = max_ts
        
        for i in range(1,8):
            if cur_ts.month in range(5, 11):
                cur_ts = cur_ts + MonthEnd(-1)
            elif cur_ts.month in (1, 4):
                cur_ts = cur_ts + MonthEnd(-3)
            else:
                print(
                    f'Fehler: Ungültige Monate in Beobachtungen.\n'
                    f'Bezirk {ID}, Eigentumsgr. {fo}, Periode {cur_ts}.\n' 
                    f'Perioden beginnend mit 02, 03, 11, 12 nicht erlaubt.\n'
                    f'Die Ausführung des Codes wird unterbrochen.'
                )
                raise SystemExit(f'Ungültige Periode {ID}, {fo}, {cur_ts}!')
                
            if (sub['ts'] == cur_ts).any():
                pass
            else:
                print(
                    f'Fehler: Benötigter Eintrag nicht gefunden.\n'
                    f'Bezirk {ID}, Eigentumsgr. {fo}, Periode {cur_ts}.\n'
                    f'Für alle Forstbezirke, für die Vorherzusagen zu treffen '
                    f'sind, werden Beobachtungen für das letzte Jahr benötigt.'
                )
                raise SystemExit(f'Fehlender Eintrag {ID}, {fo}, {cur_ts}!')

# Alle benötigten historischen Einträge gefunden


################################################################################
# Überprüfung der Vorhersageparameter
for p in [s_wt, s_du, s_kf, s_wbh]:
    if (p in range(1,100)) or (p in data['year'].unique()):
        pass
    else:
        raise SystemExit(
            'Ungültige Vorhersageparameter. Die Szenarienparameter müssen '
            'zwischen 1 und 99 liegen oder einer im Datenset enthaltenen '
            'Jahreszahl entsprechen.'
        )
        
pred_end = pd.to_datetime(pred_end) + MonthEnd()   

if pred_end - max_ts < pd.Timedelta('0'):
    raise SystemExit(
        'Ungültige Vorhersageparameter. Das Ende des Vorhersagezeitraums '
        'kann nicht in der Vergangenheit liegen.'
        )
elif pred_end - max_ts > pd.Timedelta('400 days'):
    print(
        'Warnung: Vorhersagen für die ferne Zukunft werden nicht empfohlen.'
    )
    warn_count += 1
    
################################################################################    
print(
    f'Überprüfung des Datensets und der Vorhersageparameter abgeschlossen.\n'
    f'Es traten 0 Fehler und {warn_count} Warnung(en) auf. '
)    
    

Warnung: 1276 leere Einträge in bisherigen Beobachtungen.
Dies beeinträchtigt unter Umständen die Qualität der Vorhersagen.

Überprüfung des Datensets und der Vorhersageparameter abgeschlossen.
Es traten 0 Fehler und 1 Warnung(en) auf. 


# Vorbereitung der Vorhersage

Erstelle eine neue Spalte im Datenset für die Namen der Forstbezirke. Wie auch in den vorherigen Notebooks des Projekts wird für die "alten" Bezirke mit Führungsneun der Name des aktuellen Bezirks verwendet, der diesen am besten approximiert. Diese logische Verknüpfung ist notwendig, damit der richtige Bezirk gefunden wird, wenn als Szenario ein Jahr vor 2015 verwendet wird. 

Diese und spätere Zuordnungen von Informationen erfolgen über ein Dictionary. Dies hat den Vorteil, dass im Falle von Umstrukturiereungen etc. die Werte durch den Benutzer sehr einfach "händisch" angepasst werden können. Weterhin müssen so keine zusätzlichen Daten in Google Colab eingelesen werden.

In [64]:
# Erstelle Spalte mit (neuem) Namen der Forstbezirke

# Dictionary für die Zuordnung
fdist_names = {
    2501: 'Elsterheide',
    2502: 'Bernsdorf',
    2503: 'Königswartha',
    2504: 'Nebelschütz',
    2505: 'Königsbrück',
    2506: 'Radibor',
    2507: 'Kamenz',
    2508: 'Ohorn',
    2509: 'Bischofswerda',
    2510: 'Cunewalde',
    1101: 'Chemnitz',
    1201: 'Dresden',
    2101: 'Eibenstock',
    2102: 'Zwönitz',
    2103: 'Stollberg',
    2104: 'Zschopau',
    2105: 'Annaberg',
    2106: 'Marienberg',
    2107: 'Olbernhau',
    2191: 'Eibenstock',
    2192: 'Schwarzenberg',
    2193: 'Zwönitz',
    2194: 'Stollberg',
    2195: 'Annaberg',
    2196: 'Zschopau',
    2197: 'Marienberg',
    2198: 'Olbernhau',
    2201: 'Geringswalde',
    2202: 'Striegistal',
    2203: 'Reinsberg',
    2204: 'Frauenstein',
    2601: 'Zittau',
    2602: 'Löbau',
    2603: 'Niesky',
    2604: 'Krauschwitz',
    2605: 'Boxberg',
    2606: 'Weißwasser',
    2901: 'Muldental',
    2902: 'Leipziger Land',
    2701: 'M Nord',
    2702: 'M Ost',
    2703: 'M Süd',
    2704: 'M West',
    2791: 'M Nord',
    2792: 'M Süd',
    2793: 'M Ost',
    2801: 'Freital',
    2802: 'Glashütte',
    2803: 'Bad-Gottleuba',
    2804: 'Pirna',
    2805: 'Sebnitz',
    3001: 'Delitzsch',
    3002: 'Torgau',
    3003: 'Oschatz',
    2301: 'Adorf',
    2302: 'Schöneck',
    2303: 'Weischlitz',
    2304: 'Plauen',
    2305: 'Treuen',
    2306: 'Auerbach',
    2401: 'Z Nord',
    2402: 'Z Süd',
    1302: 'Connewitz',
    1301: 'Leutzsch'
}


data['fdist_newname'] = data['fdist_id'].map(lambda x: fdist_names.get(x))


# Funktion die Skelett der neuen Vorhersage erstellt
def prepare_next(newest_ids, cur_ts):
    '''
    Diese Funktion erstellt einen neuen Dataframe für die nächste Vorhersage.
    Der Dataframe erhält die Spalten für den Forstbezirk (id, name), 
    die Eigentumsgruppe, das Jahr sowie den Zeitraum.
    
    input:
        - df: Dataframe mit Historie sowie bisherigen Vorhersagen
        
    returns:
        - Dataframe als Grundlage für nächste Vorhersage
    '''
    
    l = len(newest_ids) * 2
    
    
    df = pd.DataFrame()
    
    
    
    
# Funktion die Skelett mit features füllt
def populate_features(X, data, original_df, sc, s_wbh):

    
# Lade Modell
model = pickle.load(open('final_model.pkl', 'rb'))    

In [72]:
np.append(newest_ids, newest_ids)

array([2501, 2502, 2503, 2504, 2505, 2506, 2507, 2508, 2509, 2510, 1101,
       1201, 2101, 2102, 2103, 2104, 2105, 2106, 2107, 2201, 2202, 2203,
       2204, 2601, 2602, 2603, 2604, 2605, 2606, 2901, 2902, 2701, 2702,
       2703, 2704, 2801, 2802, 2803, 2804, 2805, 3001, 3002, 3003, 2301,
       2302, 2303, 2304, 2305, 2306, 2401, 2402, 1302, 1301, 2501, 2502,
       2503, 2504, 2505, 2506, 2507, 2508, 2509, 2510, 1101, 1201, 2101,
       2102, 2103, 2104, 2105, 2106, 2107, 2201, 2202, 2203, 2204, 2601,
       2602, 2603, 2604, 2605, 2606, 2901, 2902, 2701, 2702, 2703, 2704,
       2801, 2802, 2803, 2804, 2805, 3001, 3002, 3003, 2301, 2302, 2303,
       2304, 2305, 2306, 2401, 2402, 1302, 1301], dtype=int64)

# Vorhersagen für Szenarien berechnen

In [ ]:
original_df = data.copy()

# Vorhersagen für drei Wetterszenarien
for i in range(3):
    if i == 0:
        sc_name = 'warmtrocken'
        sc = s_wt
    elif i == 1:
        sc_name = 'gemäßigt'
        sc = s_du
    elif i == 2:
        sc_name = 'kaltfeucht'
        sc = s_kf
        
    cur_ts = max_ts     
    
    # Algorithmus ausführen, bis Ende Vorhersagezeitraum erreicht
    while cur_ts < pred_end:
        
        # Eine Periode nach vorne
        if cur_ts.month in range(4,10):
            cur_ts = cur_ts + MonthEnd(1)
        else:
            cur_ts = cur_ts + MonthEnd(3)
    
        X = prepare_next(newest_ids, cur_ts)
        X = populate_features(X, data, original_df, sc, s_wbh)
        
        # Vorhersage
        X['infested_wood'] = model.predict(X[['...', '...']])
        
        # Vorhersagen und Datenset vereinigen als Vorbereitung 
        # für nächste Iteration
        data = pd.concat(
            data, 
            X[[
                'fdist_id', 
                'year', 
                'forest_ownership', 
                'timeframe', 
                'RRK', 
                'TM0', 
                'demolition_wood', 
                'infested_wood',
                'data_fdist_newname'
            ]], 
            ignore_index=True
        )
        
    predictions = data.loc[original_df.shape[0] + 1 :]
    
    # Speichern in Excel-Datei
    predictions.to_excel('vorhersagen_'+sc_name+'.xlsx', 
                         names=[
                             ''
                         ])

    
    

In [ ]:
################################################################################ 

In [ ]:
files.download([
    'vorhersagen_warmtrocken.xlsx',
    'vorhersagen_gemäßigt.xlsx',
    'vorhersagen_kaltfeucht.xlsx',
])